In [1]:
from google.colab import files
up = files.upload()

Saving movies_dataset.csv to movies_dataset.csv


In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('movies_dataset.csv',encoding='latin1')

<ipython-input-3-0bb7ac27684f>:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('movies_dataset.csv',encoding='latin1')


In [4]:
df_in=df.reset_index()

In [5]:
df_in.head()

,index,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,...,30/10/1995,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,...,15/12/1995,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,...,22/12/1995,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,...,22/12/1995,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,...,10/02/1995,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


La siguiente función permite desanidar las colummas con listas de diccionaios para ello:
se reemplazan valores de la columna a transformar como corchetes por valores vacios.
se extraen los valores cuya clave es name, donde estan los datos importantes de la lista de diccionarios.

In [6]:
def cambio(dfr,x):
  df_2 = dfr[[x]].replace(['[]'],[""])

  df_f =df_2[x].str.extractall(r"'name': '([^']*)'").reset_index()
  df_ff = df_f[['level_0',0]]
  return df_ff.rename(columns={'level_0':'index',0:x+'dos'})

Generamos un data frame por cada columna a desanidar

In [9]:
belongs_to_collection = cambio(df_in,'belongs_to_collection')

In [12]:
genres =cambio(df_in,'genres')

In [13]:
production_companies=cambio(df_in,'production_companies')

In [14]:
production_countries=cambio(df_in,'production_countries')

In [15]:
spoken_languages =cambio(df_in,'spoken_languages')

Se unen los data frame creados con el dataframe inicial con el metood merge

In [32]:
df_1 =df_in.merge(belongs_to_collection,on='index',how='outer')

In [34]:
df_2 =df_1.merge(genres,on='index',how='outer')

In [37]:
df_3 =df_2.merge(production_companies,on='index',how='outer')

In [41]:
df_4 =df_3.merge(production_countries,on='index',how='outer')

In [42]:
df_5 =df_4.merge(spoken_languages,on='index',how='outer')

Eliminamos columnas con lista de diccionarios y otras que no se requieren

In [45]:
columnas_eliminar = ['belongs_to_collection','genres','production_companies','production_countries','spoken_languages','adult','video','imdb_id','original_title','poster_path','homepage']

In [46]:
df_final =df_5.drop(columnas_eliminar,axis=1)

las columnas revenue y budget para campos nulos los completamos con ceros

In [48]:
df_final['budget'].fillna(0,inplace=True)

In [49]:
df_final['revenue'].fillna(0,inplace=True)

Dejamos solo los campos donde release date tiene datos

In [52]:
df_f = df_final[df_final['release_date'].notnull()]

Creamos columna con return que sera el cociente entre:revenue/budget

In [61]:
df_f['budget']=pd.to_numeric(df_f['budget'], errors='coerce')

<ipython-input-61-6c41509e2aba>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_f['budget']=pd.to_numeric(df_f['budget'], errors='coerce')


In [63]:
df_f['return'] = df_f['revenue']/df_f['budget']

<ipython-input-63-7f7cac9c31b7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_f['return'] = df_f['revenue']/df_f['budget']


In [66]:
df_f['return'].fillna(0,inplace=True)

<ipython-input-66-510a4a27f051>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_f['return'].fillna(0,inplace=True)


Transformamos la columna release date a tipo date

In [ ]:
df_f.info()

In [ ]:
df_f['release_date'] = pd.to_datetime(df_f['release_date'], errors='coerce')

Se encuentran 12 valores cuyas fechas estaban en una columna incorrecta, se corrige esto

In [74]:
fechas_mal =df_f[df_f['release_date'].isnull()]

In [ ]:
fechas_mal['release_date'] = fechas_mal['id']

In [ ]:
fechas_mal['release_date'] = pd.to_datetime(fechas_mal['release_date'], errors='coerce')

In [81]:
df_f = df_f.dropna(subset = ['release_date'])

In [83]:
df_ult = pd.concat([df_f,fechas_mal])

In [86]:
df_ult['budget'].fillna(0,inplace=True)

In [89]:
df_ult['release_year'] = df_ult['release_date'].dt.year

In [93]:
df_ult['release_month'] = df_ult['release_date'].dt.month
df_ult['release_day'] = df_ult['release_date'].dt.weekday

In [95]:
df_ult.head(1)

,index,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,...,vote_average,vote_count,belongs_to_collectiondos,genresdos,production_companiesdos,production_countriesdos,spoken_languagesdos,return,release_year,release_month
0,0,30000000.0,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,...,7.7,5415.0,Toy Story Collection,Animation,Pixar Animation Studios,United States of America,English,12.451801,1995,10


In [92]:
day = {0:'Lunes',1:'Martes',2:'Miercoles',3:'Jueves',4:'Viernes',5:'Sabado',6:'Domingo'}
meses = {1:'Enero',2:'Febrero',3:'Marzo',4:'Abril',5:'Mayo',6:'Junio',7:'Julio',8:'Agosto',9:'Septiembre',10:'Octubre',11:'Noviembre',12:'Diciembre'}

In [97]:
df_ult['release_month'].replace(meses,inplace=True)
df_ult['release_day'].replace(day,inplace=True)

In [ ]:
df_ult.head(1)

In [100]:
df_ult.to_csv('movies_ultimof.csv')

Extramos directores y actores del datafram credits. El archivo credits se convierte a excel
porque se evidencia que pesa menos de esa forma.

In [ ]:
from google.colab import files
up = files.upload()

In [ ]:
df_credits = pd.read_excel('credits_2.xlsx')

In [ ]:
def cambio(dfr,x):
  df_2 = dfr[[x]].replace(['[]'],[""])

  df_f =df_2[x].str.extractall(r"'name': '([^']*)'").reset_index()
  df_ff = df_f[['level_0',0]]
  return df_ff.rename(columns={'level_0':'index',0:x+'dos'})

In [ ]:
df_credits.reset_index(inplace=True)
actores = cambio(df_credits,'cast')
actores_id =actores.merge(df_credits,on='index')
actores_final = actores_id[['castdos','id']].drop_duplicates()

Extraer directores

In [ ]:
def cambio(dfr,x):
  df_2 = dfr[[x]].replace(['[]'],[""])

  df_f =df_2[x].str.extractall(r"'Director', 'name': '([^']*)'").reset_index()
  df_ff = df_f[['level_0',0]]
  return df_ff.rename(columns={'level_0':'index',0:x+'dos'})

In [ ]:
directores = cambio(df_credits,'crew')
directores_id =directores.merge(df_credits,on='index')
directores_final = directores_id[['crewdos','id']].drop_duplicates()

convertimos columna id a int 64

In [ ]:
df_movies_id = df[df['id'].notnull()]
df_movies_id['id'] = df_movies_id['id'].astype('int64')

In [ ]:
df_actores =actores_final.merge(df_movies_id[['id','return']],on='id',how='left').drop_duplicates()
df_directores =directores_final.merge(df_movies_id[['id','return','title','release_date','revenue','budget']],on='id',how='left').drop_duplicates()

In [ ]:
df_actores.replace({np.inf:0},inplace=True)
df_directores.replace({np.inf:0},inplace=True)

Eliminamos espacios al inicio y final de las columnas crew y cast

In [ ]:
df_directores['crewdos']= df_directores['crewdos'].str.strip()
df_actores['castdos']= df_actores['castdos'].str.strip()

adicionamos la columna year para el df directores, ya que se necesitara en la api 
y se elimina la columna release date

df_directores['year'] = df_directores['release_date'].dt.year
df_directores.drop('release_date',axis=1,inplace=True)

Exportamos los archivos directores y actores.

In [ ]:
df_directores.to_excel('df_fdirector_cs.csv')
df_actores.to_excel('df_actor.xlsx')